In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler

### Google-Playstore dataset cleansing

In [ ]:
data=pd.read_csv('dataset/Google-Playstore.csv')

In [ ]:
columns = ['App Name', 'Category', 'Rating', 
'Rating Count', 'Ad Supported', 'In App Purchases', 
'Maximum Installs', 'Size', 'Released', 'Price']

g_zero = data.loc[:, columns]
g_zero.head(1)

,App Name,Category,Rating,Rating Count,Ad Supported,In App Purchases,Maximum Installs,Size,Released,Price
0,Gakondo,Adventure,0.0,0.0,False,False,15,10M,"Feb 26, 2020",0.0


In [ ]:
missing_value=g_zero.isnull().sum()
percent_missing = g_zero.isnull().sum()/g_zero.shape[0]*100

value = {
    'missing_value': missing_value,
    'percent_missing': percent_missing
}

frame = pd.DataFrame(value)
frame

,missing_value,percent_missing
App Name,2,0.000086
Category,0,0.000000
Rating,22883,0.989345
Rating Count,22883,0.989345
Ad Supported,0,0.000000
In App Purchases,0,0.000000
Maximum Installs,0,0.000000
Size,196,0.008474
Released,71053,3.071972
Price,0,0.000000


In [ ]:
g_one = g_zero.loc[:, :].dropna()
missing_value=g_one.isnull().sum()
percent_missing = g_one.isnull().sum()/g_one.shape[0]*100

value = {
    'missing_value': missing_value,
    'percent_missing': percent_missing
}

frame = pd.DataFrame(value)
frame

,missing_value,percent_missing
App Name,0,0.0
Category,0,0.0
Rating,0,0.0
Rating Count,0,0.0
Ad Supported,0,0.0
In App Purchases,0,0.0
Maximum Installs,0,0.0
Size,0,0.0
Released,0,0.0
Price,0,0.0


g_one mantengo i dati puliti, mentre g_zero, mantengo i dati "sporchi"

In [ ]:
def rewrite_byte_size(data):
    if "," in data:
        data = data.replace(",","")
    if "M" in data:
        return data.replace("M", "e6")
    elif "G" in data:
        return data.replace("G", "e9")
    elif "K" in data:
        return data.replace("K", "e3")
    elif "k" in data:
        return data.replace("k", "e3")
    elif 'Varies with device' == data:
        return data.replace(data, "5e6")
    else:
        return data

Esempio di funzionamento di rewrite_byte_size

In [ ]:
l = pd.Series(['1.K','aM','3G'])
print(l)

l = l.apply(rewrite_byte_size)
print(l)


0    1.K
1     aM
2     3G
dtype: object
0    1.e3
1     ae6
2     3e9
dtype: object


Cambiamo valori alfanumerici in numerici

In [ ]:
pd.options.mode.chained_assignment = None
g_one['Size'] = g_one['Size'].map(rewrite_byte_size)
g_one.Size.head(10)

0      10e6
1     2.9e6
2     3.7e6
3     1.8e6
4     6.2e6
5      46e6
6     2.5e6
7      16e6
9     3.5e6
10     51e6
Name: Size, dtype: object

In [ ]:
print(g_one['Size'].dtypes)
g_one['Size'] = g_one['Size'].astype(float)
print(g_one['Size'].dtypes)

object
float64


Reformattare le date in datatime, e poi in dati ordinali

In [ ]:
g_one['Released'] = pd.to_datetime(g_one['Released'], format='%b %d, %Y')
g_one['Released'].head(10)

0    2020-02-26
1    2020-05-21
2    2019-08-09
3    2018-09-10
4    2020-02-21
5    2018-12-24
6    2019-09-23
7    2019-06-21
9    2019-09-22
10   2020-07-30
Name: Released, dtype: datetime64[ns]

In [ ]:
import datetime as dt
g_one['Released'] = g_one['Released'].map(dt.datetime.toordinal)
g_one['Released'].head(10)

0     737481
1     737566
2     737280
3     736947
4     737476
5     737052
6     737325
7     737231
9     737324
10    737636
Name: Released, dtype: int64

In [ ]:
g_one.rename(columns={'Maximum Installs': 'Installs'}, inplace=True)
print(g_one['Installs'].dtypes)

int64


Le categorie sono troppe per essere visualizzate neglis step successivi, andiamo a quindi a ragrupparle in macro-categorie

In [ ]:
Education=['Education','Books & Reference','News & Magazines','Educational',
           'Art & Design','Word','Libraries & Demo','Parenting']
Tools=['Tools','Trivia','Personalization','Photography','Maps & Navigation',
       'Video Players & Editors','Weather','Communication','Finance','Business','Events']
Entertainment=['Entertainment','Music & Audio','Arcade','Puzzle','Casual','Action',
               'Simulation','Adventure','Auto & Vehicles','Board','Racing',
               'Role Playing','Strategy','Card','Casino','Music','Comics']
Lifestyle=['Lifestyle','Health & Fitness','Productivity','Shopping','Food & Drink',
           'Travel & Local','Sports','Medical','House & Home','Beauty','Dating','Social']

In [ ]:
def standardize_words(series, words_to_replace, replacement):
    return series.map(lambda x: x.replace(x, replacement) if x in words_to_replace else x)

In [ ]:
g_one.Category = standardize_words(g_one.Category, Education, 'Education')
g_one.Category = standardize_words(g_one.Category, Tools, 'Tools')
g_one.Category = standardize_words(g_one.Category, Entertainment, 'Entertainment')
g_one.Category = standardize_words(g_one.Category, Lifestyle, 'Lifestyle')

In [ ]:
g_one['Ad Supported'] = g_one['Ad Supported'].astype(int)
g_one['In App Purchases'] = g_one['In App Purchases'].astype(int)
g_one['Rating Count'] = g_one['Rating Count'].astype(int)
g_one['Installs'] = g_one['Installs'].astype(int)
print(g_one.dtypes)

App Name             object
Category             object
Rating              float64
Rating Count          int64
Ad Supported          int64
In App Purchases      int64
Installs              int64
Size                float64
Released              int64
Price               float64
dtype: object


Indice di apprezzamento = rating x rating count

In [ ]:
g_junk = g_one.copy()
g_one.drop(g_one[g_one['Installs']<10].index, inplace=True)
g_one.drop(g_one[g_one['Rating Count'] == 0].index, inplace=True)

In [ ]:
columns = ['Rating', 'Rating Count']
scaler = MinMaxScaler(feature_range=(0, 1e4))
scaled_data = scaler.fit_transform(g_one[columns])
g_one['Appreciation Index'] = scaled_data[:,0] * scaled_data[:,1]

In [ ]:
scaler = MinMaxScaler(feature_range=(0, 1e4))
scaled_data = scaler.fit_transform(g_junk[columns])
g_junk['Appreciation Index'] = scaled_data[:,0] * scaled_data[:,1]


Categorie e percentuali

In [ ]:
category_count = g_one.groupby('Category')['Category'].count()
category_percentages = category_count.transform(lambda x: 100 * x / x.sum())
print(category_count)
print(category_percentages)

Category
Education        249073
Entertainment    356054
Lifestyle        284321
Tools            313182
Name: Category, dtype: int64
Category
Education        20.710692
Entertainment    29.606280
Lifestyle        23.641602
Tools            26.041426
Name: Category, dtype: float64


In [ ]:
g_dummies = pd.get_dummies(g_one['Category'], prefix='Category')
g_columns = g_one.columns[g_one.columns != 'Category']
g_two = pd.concat([g_one[g_columns], g_dummies], axis=1)
print(g_two.dtypes)

App Name                   object
Rating                    float64
Rating Count                int64
Ad Supported                int64
In App Purchases            int64
Installs                    int64
Size                      float64
Released                    int64
Price                     float64
Appreciation Index        float64
Category_Education          uint8
Category_Entertainment      uint8
Category_Lifestyle          uint8
Category_Tools              uint8
dtype: object


In [ ]:
g_dummies = pd.get_dummies(g_junk['Category'], prefix='Category')
g_columns = g_junk.columns[g_junk.columns != 'Category']
g_junk = pd.concat([g_junk[g_columns], g_dummies], axis=1)
print(g_two.dtypes)

App Name                   object
Rating                    float64
Rating Count                int64
Ad Supported                int64
In App Purchases            int64
Installs                    int64
Size                      float64
Released                    int64
Price                     float64
Appreciation Index        float64
Category_Education          uint8
Category_Entertainment      uint8
Category_Lifestyle          uint8
Category_Tools              uint8
dtype: object


In [ ]:
%store g_two
%store g_one
%store g_junk

Stored 'g_two' (DataFrame)
Stored 'g_one' (DataFrame)
Stored 'g_junk' (DataFrame)
